In [ ]:
import os
from dotenv import load_dotenv

from langchain_community.llms import Ollama
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

load_dotenv()
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY')
## Langchain Tracking
os.environ['LANGCHAIN_API_KEY'] = os.getenv('LANGCHAIN_API_KEY')
os.environ['LANGCHAIN_TRACKING_V2'] = 'true'
os.environ['LANGCHAIN_PROJECT'] = os.getenv('LANGCHAIN_PROJECT')

from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model = 'gpt-4o')
print(llm)

client=<openai.resources.chat.completions.completions.Completions object at 0x000001F8A1441870> async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x000001F8A330CD60> root_client=<openai.OpenAI object at 0x000001F8A1D4EAD0> root_async_client=<openai.AsyncOpenAI object at 0x000001F8A330CCD0> model_name='gpt-4o' model_kwargs={} openai_api_key=SecretStr('**********') stream_usage=True


In [3]:
from langchain_community.document_loaders import WebBaseLoader

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [4]:
loader = WebBaseLoader('https://en.wikipedia.org/wiki/Large_Hadron_Collider') # It scraps data from webpage requires beautiful soup
loader

In [5]:
docs = loader.load() #loader.load() downloads the webpage and converts it into text so that LangChain can use it.
docs

[Document(metadata={'source': 'https://en.wikipedia.org/wiki/Large_Hadron_Collider', 'title': 'Large Hadron Collider - Wikipedia', 'language': 'en'}, page_content='\n\n\n\nLarge Hadron Collider - Wikipedia\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nJump to content\n\n\n\n\n\n\n\nMain menu\n\n\n\n\n\nMain menu\nmove to sidebar\nhide\n\n\n\n\t\tNavigation\n\t\n\n\nMain pageContentsCurrent eventsRandom articleAbout WikipediaContact us\n\n\n\n\n\n\t\tContribute\n\t\n\n\nHelpLearn to editCommunity portalRecent changesUpload fileSpecial pages\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nSearch\n\n\n\n\n\n\n\n\n\n\n\nSearch\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nAppearance\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nDonate\n\nCreate account\n\nLog in\n\n\n\n\n\n\n\n\nPersonal tools\n\n\n\n\n\nDonate Create account Log in\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nContents\nmove to sidebar\nhide\n\n\n\n\n(Top)\n\n\n\n\n\n1\nBackground\n\n\n\n\n\n\n\n\n2\nPurpose\n\n\n\n\n\

In [6]:
### Steps -- > Load Data --> Docs --> Divide our texts into chunks --> text --> vectors --> vector embeddings -- > vector store DB

In [7]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size = 1000,chunk_overlap=200)
documents = text_splitter.split_documents(docs)

In [8]:
documents

[Document(metadata={'source': 'https://en.wikipedia.org/wiki/Large_Hadron_Collider', 'title': 'Large Hadron Collider - Wikipedia', 'language': 'en'}, page_content='Large Hadron Collider - Wikipedia\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nJump to content\n\n\n\n\n\n\n\nMain menu\n\n\n\n\n\nMain menu\nmove to sidebar\nhide\n\n\n\n\t\tNavigation\n\t\n\n\nMain pageContentsCurrent eventsRandom articleAbout WikipediaContact us\n\n\n\n\n\n\t\tContribute\n\t\n\n\nHelpLearn to editCommunity portalRecent changesUpload fileSpecial pages\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nSearch\n\n\n\n\n\n\n\n\n\n\n\nSearch\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nAppearance\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nDonate\n\nCreate account\n\nLog in\n\n\n\n\n\n\n\n\nPersonal tools\n\n\n\n\n\nDonate Create account Log in\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nContents\nmove to sidebar\nhide\n\n\n\n\n(Top)\n\n\n\n\n\n1\nBackground\n\n\n\n\n\n\n\n\n2\nPurpose\n\n\n\n\n\n\n\n\n3

In [9]:
# We really need to convert the documents into text chunks because every LLM have some fundamental drawback of restriction in chunk size

In [10]:
from langchain_openai import OpenAIEmbeddings
embeddings = OpenAIEmbeddings()
## here we will be converting our documents into numbers vectors using openai embeddings


vectorstoredb = FAISS.from_documents(documents,embeddings)


In [11]:
from langchain_community.vectorstores import Chroma
db = Chroma.from_documents(documents, embedding=embeddings, collection_name="my_docs")


In [22]:
#retriever = db.as_retriever()
retriever = db.as_retriever(search_kwargs={"k": 5})
docs1 = retriever.invoke('The LHC first went operational on 10 September 2008,[56] but initial testing was delayed for 14 months from 19 September 2008 to 20 November 2009, following a magnet quench incident that caused extensive damage to over 50 superconducting magnets, their mountings, and the vacuum pipe')
docs1


[Document(metadata={'language': 'en', 'source': 'https://en.wikipedia.org/wiki/Large_Hadron_Collider', 'title': 'Large Hadron Collider - Wikipedia'}, page_content='In the original schedule for LHC commissioning, the first "modest" high-energy collisions at a centre-of-mass energy of 900\xa0GeV were expected to take place before the end of September 2008, and the LHC was expected to be operating at 10\xa0TeV by the end of 2008.[92] However, owing to the delay caused by the incident, the collider was not operational until November 2009.[93] Despite the delay, LHC was officially inaugurated on 21\xa0October 2008, in the presence of political leaders, science ministers from CERN\'s 20 Member States, CERN officials, and members of the worldwide scientific community.[94]\nMost of 2009 was spent on repairs and reviews from the damage caused by the quench incident, along with two further vacuum leaks identified in July 2009; this pushed the start of operations to November of that year.[79]'),


db → your database

retriever → a smart search engine built on top of that database

The retriever makes it much easier to plug into LangChain’s RAG system.

In [13]:
from langchain_community.llms import Ollama
llm = Ollama(model="gemma:2b")


C:\Users\kriti\AppData\Local\Temp\ipykernel_26356\752346876.py:2: LangChainDeprecationWarning: The class `Ollama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the `langchain-ollama package and should be used instead. To use it run `pip install -U `langchain-ollama` and import as `from `langchain_ollama import OllamaLLM``.
  llm = Ollama(model="gemma:2b")


In [ ]:
def answer_question(query):
    # Retrieve relevant chunks
    docs = retriever.invoke(query)

    # Build context string
    context = "\n\n".join([doc.page_content for doc in docs])

    # Build prompt
    prompt = f"""
    You are a helpful and accurate assistant.  
    Use ONLY the provided context to answer the question.  
    If the answer is not found, say "The document does not contain this information."

    Context:
    {context}

    Question: {query}

    Answer:
    """

    # LLM inference (OpenAI or Ollama)
    return llm.invoke(prompt)



In [24]:
response = answer_question("What is  Large Hadron Collidor? and where is it located. what is the physics being used there")
print(response)




Sure. Here's the answer to the question:

**What is the Large Hadron Collider?**
The Large Hadron Collider (LHC) is the world's largest and highest-energy particle accelerator. It was built by the European Organization for Nuclear Research (CERN) between 1998 and 2008 in collaboration with over 10,000 scientists, and hundreds of universities and laboratories across more than 100 countries.

**Where is it located?**
The LHC lies in a tunnel 27 kilometres (17 mi) in circumference and as deep as 175 metres (574 ft) beneath the France–Switzerland border near Geneva.

**What is the physics being used there?**
The LHC uses particle accelerators to explore and study the fundamental laws governing the interactions and forces among elementary particles and the deep structure of space and time.


In [ ]:
#from similarity search algo . They compute cosine similarity between query vector and document vectors. 
# Return the top-k most similar documents
#Retrieval = the process of fetching relevant documents using similarity search

In [17]:
query = "The beams intersect at four points around the ring, which is where the particle collisions take place. Some 1,232 dipole magnets keep the beams on their circular path (see image[32]), while an additional 392 quadrupole magnets are used to keep the beams focused, with stronger quadrupole magnets close to the intersection points in order to maximize the chances of interaction where the two beams cross."
result = db.similarity_search(query)
result[0].page_content

'The collider tunnel contains two adjacent parallel beamlines (or beam pipes) each containing a beam, which travel in opposite directions around the ring. The beams intersect at four points around the ring, which is where the particle collisions take place. Some 1,232 dipole magnets keep the beams on their circular path (see image[32]), while an additional 392 quadrupole magnets are used to keep the beams focused, with stronger quadrupole magnets close to the intersection points in order to maximize the chances of interaction where the two beams cross. Magnets of higher multipole orders are used to correct smaller imperfections in the field geometry. In total, about 10,000 superconducting magnets are installed,[33] with each of the 1232 dipole magnets having a mass of 35 tonnes.[34] About 96 tonnes of superfluid helium-4 is needed to keep the magnets, made of copper-clad niobium-titanium, at their operating temperature of 1.9\xa0K (−271.25\xa0°C), making the LHC the largest cryogenic'